# Task
Build an AI contextual legal case summary tool with the following specifications:
- **Prompt Template**: Act as a 42-year-old IP consultant mentoring a junior consultant. Include sections for "Background," "Ruling," and "Relevance to Context." Incorporate proper legal sections, laws, acts, previous cases, and judgments.
- **AI Model Configuration**: Set the AI model temperature between 0 and 0.3.
- **Output Format**:
    - Token length: 300-480 tokens.
    - Organized into three sections:
        - Background: The facts and context of the case.
        - Ruling: The court’s decision and reasoning.
        - Relevance to Context: How the case relates to the provided context.
- **Guardrails**: Implement guardrails to ensure the output adheres to the specified format and constraints.

## Setup

### Subtask:
Install and import necessary libraries.


**Reasoning**:
Install the required library using pip.



In [1]:
%pip install -U google-generativeai

**Reasoning**:
Import the installed library to make it available for use.



In [2]:
import google.generativeai as genai

## Configure ai model

### Subtask:
Configure the AI model with the specified temperature and token length.


**Reasoning**:
I will now configure the generative AI model with the specified parameters. This involves setting up the API key, defining the generation configuration with the required temperature and token limits, and then instantiating the generative model. I will use a placeholder for the API key as instructed, and choose values within the specified ranges for temperature and token count.



In [3]:
genai.configure(api_key="YOUR_API_KEY")

generation_config = genai.GenerationConfig(
    temperature=0.2,
    max_output_tokens=400,
)

model = genai.GenerativeModel(
    model_name='gemini-1.5-flash',
    generation_config=generation_config
)

## Define prompt template

### Subtask:
Create a detailed prompt template incorporating the persona, required sections, and legal elements.


**Reasoning**:
Define a multi-line string variable to hold the prompt template, incorporating the persona, required sections, legal elements, and placeholders for the case data.



In [4]:
prompt_template = """
Alright junior, let's dive into this case. As your mentor, a seasoned 42-year-old IP consultant, I'll guide you on how to dissect legal cases effectively. We need to summarize this case with our current project context in mind.

Here's the legal case data:
Case Facts: {case_facts}
Case Ruling: {case_ruling}
Context: {context}

Your task is to summarize this case, focusing on its relevance to our context. Structure your summary into the following three sections:

Background:
Provide the essential facts and the initial context of the case. Ensure you incorporate relevant legal sections, applicable laws, acts, and mention any significant previous cases or judgments that set the stage.

Ruling:
Clearly explain the court's decision and the reasoning behind it. Again, include references to the specific legal sections, laws, acts, and any prior judgments that influenced this ruling.

Relevance to Context:
Explain how this case and its ruling are pertinent to the context provided. Connect the legal principles and outcomes of this case to our current situation.

Remember to be concise and focus on the information most relevant to our IP consulting work.
"""

## Implement guardrails

### Subtask:
Add guardrails to ensure the output adheres to the specified format and constraints.


**Reasoning**:
Define a function to apply the guardrails to the generated text. This function will check for the presence of the required sections and the token length.



In [5]:
import tiktoken

def enforce_guardrails(generated_text, min_tokens=300, max_tokens=480):
    """
    Checks if the generated text adheres to the specified format and constraints.

    Args:
        generated_text: The text generated by the AI model.
        min_tokens: The minimum allowed token length.
        max_tokens: The maximum allowed token length.

    Returns:
        True if the text adheres to the constraints, False otherwise.
    """
    # Check for required sections
    required_sections = ["Background:", "Ruling:", "Relevance to Context:"]
    for section in required_sections:
        if section not in generated_text:
            print(f"Missing section: {section}")
            return False

    # Check token length
    encoding = tiktoken.get_encoding("cl100k_base")
    token_count = len(encoding.encode(generated_text))

    if not (min_tokens <= token_count <= max_tokens):
        print(f"Token count out of range: {token_count} tokens")
        return False

    return True


**Reasoning**:
The guardrail function is defined. The next step is to integrate this function into the process of generating the summary and add logic to regenerate if needed. Since the prompt template and model are already defined, the next logical step is to create a function that uses these components, generates the text, and applies the guardrails, including regeneration logic.



In [6]:
def generate_legal_summary(model, prompt_template, case_facts, case_ruling, context, max_retries=3):
    """
    Generates a legal case summary using the AI model and enforces guardrails.

    Args:
        model: The generative AI model.
        prompt_template: The template for the prompt.
        case_facts: The facts of the case.
        case_ruling: The ruling of the case.
        context: The context for relevance.
        max_retries: The maximum number of regeneration attempts.

    Returns:
        The generated summary if it passes guardrails, or None after max_retries.
    """
    prompt = prompt_template.format(case_facts=case_facts, case_ruling=case_ruling, context=context)

    for attempt in range(max_retries):
        try:
            response = model.generate_content(prompt)
            generated_text = response.text

            if enforce_guardrails(generated_text):
                print(f"Summary generated successfully after {attempt + 1} attempts.")
                return generated_text
            else:
                print(f"Guardrail check failed on attempt {attempt + 1}. Regenerating...")

        except Exception as e:
            print(f"An error occurred during generation on attempt {attempt + 1}: {e}")
            # Optionally, add logic to modify prompt or parameters based on error type
            pass # Continue to next attempt

    print(f"Failed to generate a valid summary after {max_retries} attempts.")
    return None

## Test the tool

### Subtask:
Test the complete setup with example legal case data to ensure it produces summaries as expected.


**Reasoning**:
Define example variables for case_facts, case_ruling, and context and then call the generate_legal_summary function with the defined variables and print the result.



In [7]:
case_facts = """
In the case of Apple Inc. v. Samsung Electronics Co., Ltd., the dispute involved allegations of patent infringement regarding smartphone technology. Apple claimed Samsung's Galaxy line of products infringed upon Apple's patents related to design and utility features of the iPhone. The case was filed in the United States District Court for the Northern District of California.
"""

case_ruling = """
The court found Samsung liable for infringing several of Apple's design and utility patents. A jury awarded Apple over $1 billion in damages, which was later reduced after appeals. The ruling affirmed the protectability of smartphone design and functionality under existing patent law, including elements like the rounded corners and grid of icons.
"""

context = """
Our client is developing a new mobile device and wants to ensure they avoid infringing on existing patents, particularly in the areas of user interface design and hardware features similar to those involved in the Apple v. Samsung case. We need to understand the key aspects of that ruling to advise them properly.
"""

summary = generate_legal_summary(model, prompt_template, case_facts, case_ruling, context)

if summary:
    print(summary)
else:
    print("Failed to generate a valid summary.")

An error occurred during generation on attempt 1: 400 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint: API key not valid. Please pass a valid API key.


An error occurred during generation on attempt 2: 400 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint: API key not valid. Please pass a valid API key.
An error occurred during generation on attempt 3: 400 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint: API key not valid. Please pass a valid API key.
Failed to generate a valid summary after 3 attempts.
Failed to generate a valid summary.


**Reasoning**:
The previous command failed because the API key was not valid. I need to update the API key and re-run the test with example data.



In [8]:
genai.configure(api_key="YOUR_API_KEY")

case_facts = """
In the case of Apple Inc. v. Samsung Electronics Co., Ltd., the dispute involved allegations of patent infringement regarding smartphone technology. Apple claimed Samsung's Galaxy line of products infringed upon Apple's patents related to design and utility features of the iPhone. The case was filed in the United States District Court for the Northern District of California.
"""

case_ruling = """
The court found Samsung liable for infringing several of Apple's design and utility patents. A jury awarded Apple over $1 billion in damages, which was later reduced after appeals. The ruling affirmed the protectability of smartphone design and functionality under existing patent law, including elements like the rounded corners and grid of icons.
"""

context = """
Our client is developing a new mobile device and wants to ensure they avoid infringing on existing patents, particularly in the areas of user interface design and hardware features similar to those involved in the Apple v. Samsung case. We need to understand the key aspects of that ruling to advise them properly.
"""

summary = generate_legal_summary(model, prompt_template, case_facts, case_ruling, context)

if summary:
    print(summary)
else:
    print("Failed to generate a valid summary.")

An error occurred during generation on attempt 1: 400 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint: API key not valid. Please pass a valid API key.


An error occurred during generation on attempt 2: 400 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint: API key not valid. Please pass a valid API key.
An error occurred during generation on attempt 3: 400 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint: API key not valid. Please pass a valid API key.
Failed to generate a valid summary after 3 attempts.
Failed to generate a valid summary.


## Test the tool retry 1

### Subtask:
Test the complete setup with example legal case data to ensure it produces summaries as expected. This is a retry of the previous failed attempt.


**Reasoning**:
Define example legal case data and call the `generate_legal_summary` function to test the complete setup. Check if the summary is not None and print it or a failure message. Ensure the API key is set correctly within the code block.



In [9]:
genai.configure(api_key="YOUR_API_KEY")

case_facts = """
In the case of Apple Inc. v. Samsung Electronics Co., Ltd., the dispute involved allegations of patent infringement regarding smartphone technology. Apple claimed Samsung's Galaxy line of products infringed upon Apple's patents related to design and utility features of the iPhone. The case was filed in the United States District Court for the Northern District of California.
"""

case_ruling = """
The court found Samsung liable for infringing several of Apple's design and utility patents. A jury awarded Apple over $1 billion in damages, which was later reduced after appeals. The ruling affirmed the protectability of smartphone design and functionality under existing patent law, including elements like the rounded corners and grid of icons.
"""

context = """
Our client is developing a new mobile device and wants to ensure they avoid infringing on existing patents, particularly in the areas of user interface design and hardware features similar to those involved in the Apple v. Samsung case. We need to understand the key aspects of that ruling to advise them properly.
"""

# Ensure that the generate_legal_summary function and model are defined from previous steps
# and that the prompt_template is also defined.

summary = generate_legal_summary(model, prompt_template, case_facts, case_ruling, context)

if summary:
    print(summary)
else:
    print("Failed to generate a valid summary.")

An error occurred during generation on attempt 1: 400 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint: API key not valid. Please pass a valid API key.


An error occurred during generation on attempt 2: 400 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint: API key not valid. Please pass a valid API key.
An error occurred during generation on attempt 3: 400 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint: API key not valid. Please pass a valid API key.
Failed to generate a valid summary after 3 attempts.
Failed to generate a valid summary.


## Summary:

### Data Analysis Key Findings

*   The necessary libraries (`google-generativeai` and `tiktoken`) for building the AI tool were successfully installed and imported.
*   The AI model (`gemini-1.5-flash`) was configured with the specified temperature (0.2) and maximum output tokens (400).
*   A detailed prompt template was created, incorporating the persona of a 42-year-old IP consultant, defining the required output sections ("Background," "Ruling," and "Relevance to Context"), and instructing the inclusion of legal elements.
*   Guardrails were implemented through Python functions (`enforce_guardrails` and `generate_legal_summary`) to check for required sections and token length (300-480 tokens) and include a retry mechanism for generation.
*   Testing of the complete setup with example data failed because a valid API key for the generative AI model was not provided, preventing successful authentication and generation of the summary.

### Insights or Next Steps

*   The primary next step is to provide a valid API key for the `google-generativeai` service to enable successful testing and operation of the legal case summary tool.
*   Once a valid API key is provided, re-run the test case to confirm that the AI model generates summaries adhering to the defined prompt template, format, token length, and guardrail requirements.
